# Broadcase Variables

In [ ]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Broadcast Variable") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

In [ ]:
# Our example dataset

_data = [
    ["Ramesh", "D001", "Apache Spark"],
    ["Siv", "D001", "C++"],
    ["Imran", "D002", "English"],
    ["Akshay", "D003", "Hindi"],
    ["Somesh", "D002", "Scala"],
    ["Hitesh", "D001", "Physics"]
]

_cols = ["NAME", "DEPT_CODE", "FAV_SUBJECT"]

df_students = spark.createDataFrame(data=_data, schema=_cols)
df_students.printSchema()
df_students.show()s

In [ ]:
# Broadcast Varible

dept_names = {"D001": "Department 1", "D002": "Department 2", "D003": "Department 3"}
dept_est = {"D001": 1990, "D003": 2001}

broadcast_dept_names = spark.sparkContext.broadcast(dept_names)
broadcast_dept_est = spark.sparkContext.broadcast(dept_est)

# Check the type of the variable
print("The type is: " + str(type(broadcast_dept_est)))

# In case we want to check the value
print(broadcast_dept_names.value)
print(broadcast_dept_est.value)

In [ ]:
# Lets use broadcast variable to use Dept info
_new_schema = ["NAME", "DEPT_CODE", "FAV_SUBJECT", "DEPT_NAME", "ESTD"]

# Use lambda function to iterate over row to get the broadcast value
df = df_students.rdd.map(lambda row: [
    row.NAME, 
    row.DEPT_CODE, 
    row.FAV_SUBJECT,
    broadcast_dept_names.value.get(row.DEPT_CODE),
    broadcast_dept_est.value.get(row.DEPT_CODE)]).toDF(_new_schema)

df.show()